# Whisper Fine-Tuning with Weights & Biases (WandB)

This notebook includes:
1.  **WandB Integration**: For tracking loss, WER, and system metrics (GPU usage).
2.  **Robust Data Prep**: Handles missing files and language filtering automatically.
3.  **A40 Optimization**: High batch size, FP16, and disabled gradient checkpointing for stability.

In [ ]:
# --- STEP 1: Install Dependencies ---
!pip install datasets transformers torchaudio evaluate jiwer torchcodec audiomentations tensorboard scikit-learn accelerate wandb
!pip install --upgrade transformers accelerate

In [ ]:
# --- STEP 2: Imports & WandB Login ---
import os
import wandb
from huggingface_hub import login

# Log in to Weights & Biases
# You will be prompted to enter your API key (get it from https://wandb.ai/authorize)
wandb.login()

# Optional: Set environment variables to keep the run quiet or offline if needed
# os.environ["WANDB_SILENT"] = "true"

In [ ]:
# --- STEP 3: Dataset Preparation ---
import pandas as pd
import torchaudio
import torch
from datasets import Dataset, Audio
from sklearn.model_selection import train_test_split
from transformers import WhisperProcessor

# Config
audio_base_path = "/workspace/data/audio_train/librivox-indonesia"
metadata_path = os.path.join(audio_base_path, "metadata_train_wav.csv")
MODEL_NAME = "openai/whisper-small"
TARGET_LANGUAGE = "min"  # Target language code
SAMPLING_RATE = 16000

# 1. Load Metadata
if not os.path.exists(metadata_path):
    raise FileNotFoundError(f"❌ Metadata file not found at {metadata_path}.")

print(f"📂 Loading metadata from: {metadata_path}")
df = pd.read_csv(metadata_path)

# 2. Filter Language (with Fallback)
if "language" not in df.columns:
    raise ValueError(f"❌ Column 'language' not found! Available: {df.columns}")

df_lang = df[df["language"] == TARGET_LANGUAGE].copy()

if len(df_lang) == 0:
    print(f"⚠️ No samples found for '{TARGET_LANGUAGE}'. Switching to most common language.")
    most_common = df["language"].mode()[0]
    df_lang = df[df["language"] == most_common].copy()
    print(f"🔄 Switched to: '{most_common}' ({len(df_lang)} samples)")

# 3. Cleanup Paths
df_lang = df_lang.dropna(subset=["path"])
df_lang["path"] = df_lang["path"].astype(str)
df_lang["full_path"] = df_lang["path"].apply(lambda p: os.path.join(audio_base_path, p))

# Verify files exist
df_lang["exists"] = df_lang["full_path"].apply(os.path.exists)
df_lang = df_lang[df_lang["exists"]]

print(f"✅ Final Dataset Size: {len(df_lang)} samples")

# 4. Split
train_df, val_df = train_test_split(df_lang, test_size=0.1, random_state=42)
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

# 5. Processor & Mapping
print("⚙️ Loading Processor...")
# Use 'indonesian' token for better transfer learning on local languages
processor = WhisperProcessor.from_pretrained(MODEL_NAME, language="indonesian", task="transcribe")

def prepare_dataset(batch):
    audio = torchaudio.load(batch["full_path"])[0]
    # Resample/Mono logic handled by torchaudio load usually, but ensuring safety:
    # (Simplified for brevity, assuming 16khz/mono from previous conversion step)
    if audio.shape[0] > 1: audio = audio.mean(dim=0, keepdim=True)
    
    inputs = processor(
        audio=audio.squeeze().numpy(),
        sampling_rate=SAMPLING_RATE,
        text=batch["sentence"]
    )
    return {"input_features": inputs["input_features"][0], "labels": inputs["labels"]}

print("🔄 Processing dataset...")
train_ds = train_ds.map(prepare_dataset, num_proc=1)
val_ds = val_ds.map(prepare_dataset, num_proc=1)

print("✅ Data ready.")

In [ ]:
# --- STEP 4: Metrics & Data Collator ---
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

@torch.no_grad()
def data_collator(features):
    input_features = [{"input_features": f["input_features"]} for f in features]
    labels = [f["labels"] for f in features]

    batch = processor.feature_extractor.pad(input_features, return_tensors="pt")
    
    # Manually pad labels
    max_len = max(len(l) for l in labels)
    padded_labels = [l + [-100]*(max_len - len(l)) for l in labels]
    batch["labels"] = torch.tensor(padded_labels, dtype=torch.long)
    
    # Trim BOS
    if (batch["labels"][:, 0] == processor.tokenizer.bos_token_id).all():
        batch["labels"] = batch["labels"][:, 1:]
        
    return batch

In [ ]:
# --- STEP 5: Model Loading & Training ---
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import gc

# Cleanup memory
if 'model' in globals(): del model
if 'trainer' in globals(): del trainer
gc.collect()
torch.cuda.empty_cache()

print("🚀 Loading Model...")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model.to("cuda")
model.config.use_cache = False # Disable cache for training

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-minang-wandb",
    
    # WandB Configuration
    report_to="wandb",                # <--- ENABLE WANDB HERE
    run_name="whisper-minang-run-01", # Name your run
    
    # A40 Optimization
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=1000,
    gradient_checkpointing=False,     # DISABLED for stability on A40
    fp16=True,
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=25,
    predict_with_generate=True,
    generation_max_length=225,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    processing_class=processor,
    compute_metrics=compute_metrics,  # Metric function attached
)

print("✅ Starting Training with WandB logging...")
trainer.train()

In [ ]:
# --- STEP 6: Finish Run ---
# Mark the run as finished in WandB
wandb.finish()